#### Libraries

In [1]:
%%javascript
utils.load_extension('collapsible_headings/main')
utils.load_extension('hide_input/main')
utils.load_extension('autosavetime/main')
utils.load_extension('execute_time/ExecuteTime')
utils.load_extension('code_prettify/code_prettify')
utils.load_extension('scroll_down/main')
utils.load_extension('jupyter-js-widgets/extension')

<IPython.core.display.Javascript object>

In [2]:
from sklearn import *
import sklearn
import time
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
import joblib
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn import pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


from sklearn.feature_selection import VarianceThreshold
import zipfile
import os

from lightgbm import LGBMRegressor


import random
random.seed(0)

from tqdm import tqdm
import time

from category_encoders.target_encoder import TargetEncoder
from category_encoders.m_estimate import MEstimateEncoder
from category_encoders.utils import TransformerWithTargetMixin

import warnings
warnings.filterwarnings('ignore')

import sktools

from sktools import QuantileEncoder

from tabulate import tabulate

/Users/davidmasip/opt/anaconda3/envs/sk-experiments/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [3]:
class TypeSelector(BaseEstimator, TransformerMixin):
    '''
    Transformer that filters a type of columns of a given data frame.
    '''
    def __init__(self, dtype):
        self.dtype = dtype
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        #print("Type Selector out shape {}".format(X.select_dtypes(include=[self.dtype]).shape))
        #print(X.select_dtypes(include=[self.dtype]).dtypes)
        return X.select_dtypes(include=[self.dtype])

def elapsed_time_mins (time1,time2):
    elapsed = np.round(np.abs(time1-time2)/60,decimals=2)

    return elapsed

class SummaryEncoder(BaseEstimator, TransformerWithTargetMixin):

    def __init__(self, cols, quantiles, m=1.):

        self.cols = cols
        self.quantiles = quantiles
        self.m = m
        self.encoder_list = None

    def fit(self, X, y):
        
        X = X.copy()
        
        for quantile in self.quantiles:
            for col in self.cols:
                percentile = round(quantile * 100)
                X[f'{col}_{percentile}'] = X[col]

        encoder_list = []
        for quantile in self.quantiles:
            col_names = []
            for col in self.cols:
                percentile = round(quantile * 100)
                col_names.append(f'{col}_{percentile}')
            enc = QuantileEncoder(
                cols=col_names, quantile=quantile, m=self.m
            )
            enc.fit(X, y)
            encoder_list.append(enc)

        self.encoder_list = encoder_list

        return self

    def transform(self, X, y=None):
        X_encoded = X.copy()
        
        for quantile in self.quantiles:
            for col in self.cols:
                percentile = round(quantile * 100)
                X_encoded[f'{col}_{percentile}'] = X_encoded[col]

        for encoder in self.encoder_list:
            X_encoded = encoder.transform(X_encoded)
        return X_encoded



def fit_pipe(pipe,pipe_grid,X,y,subsample=False,n_max=20_000,best_params=True):
    
    if subsample:
        X = X[0:n_max]
        y = y[0:n_max]
    
    # Instantiate the grid
    pipe_cv = GridSearchCV(pipe, param_grid=pipe_grid, n_jobs = -1, cv=cv, scoring="neg_mean_absolute_error")
    
    pipe_cv.fit(X,y)
    
    best_estimator = pipe_cv.best_estimator_.fit( X_tr, y_tr)
    grid_results = pd.DataFrame(pipe_cv.cv_results_)
    
    return best_estimator,grid_results,pipe_cv.best_params_

## Define the data

d = pd.read_csv('data/stackoverflow.csv')

d.ConvertedSalary = pd.to_numeric(d.ConvertedSalary,errors='coerce')

d = d[d.ConvertedSalary.isna()!=True]



d.to_csv('data/stackoverflow_clean.csv',index=False)

In [4]:
data = [
    'data/house_kaggle.csv',
    'data/stackoverflow.csv',
    'data/ks.csv',
    'data/medical_payments_sample.csv',
    'data/cauchy.csv'
]

In [5]:
drop = [
    ['Id','BsmtQual', 'BsmtCond','BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2','BsmtFinSF2', 'BsmtUnfSF','LowQualFinSF','FullBath','HalfBath'],
    ['Respondent','Salary'],
    [],
    ['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name','Number_of_Payments_Included_in_Total_Amount'],
    []
]

In [6]:
cols_enc = [
    ['MSSubClass','MSZoning','LotShape','LandContour','Utilities','LotConfig','Neighborhood','BldgType','HouseStyle','YearBuilt','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','ExterQual','MasVnrType','Heating','HeatingQC'],
    ['Country','Employment','FormalEducation','UndergradMajor','CompanySize','DevType','YearsCoding','LanguageWorkedWith','LanguageDesireNextYear','RaceEthnicity'],
    ['category', 'main_category', 'currency','state','country'],
    
    ['Recipient_City', 'Recipient_State', 'Recipient_Zip_Code','Recipient_Country', 'Physician_Primary_Type',
       'Physician_License_State_code1',
       'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country',
       'Form_of_Payment_or_Transfer_of_Value','Nature_of_Payment_or_Transfer_of_Value'],
    
    ['value_1', 'value_2']
]

In [7]:
target = [
    ['SalePrice'],
    ['ConvertedSalary'],
    ['goal'],
    ['Total_Amount_of_Payment_USDollars'],
    ['target']
]

# Loop

In [8]:
n_jobs = -1
float_eltype = np.float32
resultados = []
tic=time.time()

n_max = 20_000
cv = 4
filter_size = 2_000
columns =['NameDataset',
          # Scores
          'enet_te_train_mae','enet_te_test_mae',
          'enet_te_train_mse','enet_te_test_mse',
          
          'enet_pe_train_mae','enet_pe_test_mae',
          'enet_pe_train_mse','enet_pe_test_mse',
          
          'xgb_te_train_mae','xgb_te_test_mae',
          'xgb_te_train_mse','xgb_te_test_mse',
          
          'xgb_pe_train_mae','xgb_pe_test_mae',
          'xgb_pe_train_mse','xgb_pe_test_mse',
          
          
          'size',
          
          # Params
          'enet_te_best_params','enet_pe_best_params',
          # Time
          'time_train_m']    


In [9]:



print(tabulate(tabular_data=[], headers=['Data', 'Model', 'Train', 'Test'], tablefmt="psql"))

for i in range(0,len(data)):
    
    # Read data
    df = pd.read_csv(data[i])
    if df.shape[0] > 100_000:
        df = df.sample(n=100_000)

    
    # Drop columns 
    df = df.drop(columns=drop[i])
    

    # Fillna
    df.fillna(0,inplace=True)
    
    print(df.shape)
    # Train-Test Split
    X_tr, X_te, y_tr, y_te = sklearn.model_selection.train_test_split(df.drop(columns=target[i]), df[target[i]])
   


    # Elastic Net + target encoding
    scaler  = sklearn.preprocessing.StandardScaler()
    clf = sklearn.linear_model.ElasticNet()
    te = MEstimateEncoder(cols=cols_enc[i])
        

    pipe = Pipeline([
            ('te',te),
            ('selector', TypeSelector(np.number)), # Selects Numerical Columns only
            ('scaler', scaler),
            ('clf',clf )])
        
    pipe_grid = {
        "te__m":[1],
    }
    
    # Train model
    enet_te,enet_te_grid_results,enet_te_params = fit_pipe(pipe,pipe_grid,X_tr,y_tr)


    score_enet_te_train = mean_absolute_error(y_tr, enet_te.predict(X_tr))
    score_enet_te_test = mean_absolute_error(y_te, enet_te.predict(X_te))
    
    score_enet_te_train_mse = mean_squared_error(y_tr, enet_te.predict(X_tr))
    score_enet_te_test_mse = mean_squared_error(y_te, enet_te.predict(X_te))

    print(tabulate(tabular_data=[[data[i][5:10], 'enet_te', score_enet_te_train, score_enet_te_test]], tablefmt='psql'))

    
    
    # Elastic Net + percentile encoding
    scaler  = sklearn.preprocessing.StandardScaler()
    clf = sklearn.linear_model.ElasticNet()
    pe = SummaryEncoder(cols= cols_enc[i],quantiles=[.25, .50, .75])
        

    pipe = Pipeline([
            ('pe',pe),
            ('selector', TypeSelector(np.number)), # Selects Numerical Columns only
            ('scaler', scaler),
            ('clf',clf )])
        
    pipe_grid = { 

        }
    
    # Train model
    enet_pe,enet_pe_grid_results,enet_pe_params = fit_pipe(pipe,pipe_grid,X_tr,y_tr)


    score_enet_pe_train = mean_absolute_error(y_tr, enet_pe.predict(X_tr))
    score_enet_pe_test = mean_absolute_error(y_te, enet_pe.predict(X_te))
    
    score_enet_pe_train_mse = mean_squared_error(y_tr, enet_pe.predict(X_tr))
    score_enet_pe_test_mse = mean_squared_error(y_te, enet_pe.predict(X_te))
    print(tabulate(tabular_data=[[data[i][5:10], 'enet_pe', score_enet_pe_train,score_enet_pe_test]], tablefmt='psql'))
    
        
        
        
        
        
        
        
        
    # xgb + target encoding
    scaler  = sklearn.preprocessing.StandardScaler()
    clf = LGBMRegressor()
    te = MEstimateEncoder(cols=cols_enc[i])
    var = VarianceThreshold(threshold=0.1)
        

    pipe = Pipeline([
            ('te',te),
            ('selector', TypeSelector(np.number)), # Selects Numerical Columns only
            ('var',var),
            ('scaler', scaler),
            ('clf',clf )])
        
    pipe_grid = {
        "te__m":[1],
    }
    

    # Train model
    xgb_te,xgb_te_grid_results,xgb_te_params = fit_pipe(pipe,pipe_grid,X_tr,y_tr)


    score_xgb_te_train = mean_absolute_error(y_tr, xgb_te.predict(X_tr))
    score_xgb_te_test = mean_absolute_error(y_te, xgb_te.predict(X_te))
    
    score_xgb_te_train_mse = mean_squared_error(y_tr, xgb_te.predict(X_tr))
    score_xgb_te_test_mse = mean_squared_error(y_te, xgb_te.predict(X_te))

    print(tabulate(tabular_data=[[data[i][5:10], 'xgbs_te ', score_xgb_te_train,score_xgb_te_test]], tablefmt='psql'))
    
        
    
    
    # xgb + percentile encoding
    scaler  = sklearn.preprocessing.StandardScaler()
    clf = LGBMRegressor()
    pe = SummaryEncoder(cols= cols_enc[i],quantiles=[.25, .50, .75])
    var = VarianceThreshold(threshold=0.01)
        

    pipe = Pipeline([
            ('pe',pe),
            ('selector', TypeSelector(np.number)), # Selects Numerical Columns only
            ('var',var),
            ('scaler', scaler),
            ('clf',clf )])
        
    pipe_grid = { 

        }
    
    # Train model
    xgb_pe,xgb_pe_grid_results,xgb_pe_params = fit_pipe(pipe,pipe_grid,X_tr,y_tr)


    score_xgb_pe_train = mean_absolute_error(y_tr, xgb_pe.predict(X_tr))
    score_xgb_pe_test = mean_absolute_error(y_te, xgb_pe.predict(X_te))
    
    score_xgb_pe_train_mse = mean_squared_error(y_tr, xgb_pe.predict(X_tr))
    score_xgb_pe_test_mse = mean_squared_error(y_te, xgb_pe.predict(X_te))
    
    print(tabulate(tabular_data=[[data[i][5:10], 'xgbs_pe', score_xgb_pe_train,score_xgb_pe_test]], tablefmt='psql'))
    
    
    
    
    # Grid Results
    pd.DataFrame(enet_te_grid_results).to_csv('./results_regression/grid_results/{}_{}.csv'.format('enet_te_grid_results',data[i][5:10]))
    pd.DataFrame(enet_pe_grid_results).to_csv('./results_regression/grid_results/{}_{}.csv'.format('enet_pe_grid_results',data[i][5:10]))
    pd.DataFrame(xgb_te_grid_results).to_csv('./results_regression/grid_results/{}_{}.csv'.format('xgb_te_grid_results',data[i][5:10]))
    pd.DataFrame(xgb_pe_grid_results).to_csv('./results_regression/grid_results/{}_{}.csv'.format('xgbt_pe_grid_results',data[i][5:10]))

    
    
    # Add Results
    resultados.append([data[i].split('/')[1],
                       #Scores
                       score_enet_te_train,score_enet_te_test,
                       score_enet_te_train_mse,score_enet_te_test_mse,
                       
                       score_enet_pe_train,score_enet_pe_test,
                       score_enet_pe_train_mse,score_enet_pe_test_mse,
                       
                       score_xgb_te_train,score_xgb_te_test,
                       score_xgb_te_train_mse,score_xgb_te_test_mse,
                       
                       score_xgb_pe_train,score_xgb_pe_test,
                       score_xgb_pe_train_mse,score_xgb_pe_test_mse,
                       
                       # Shape
                       df.shape,
                       
                       # params
                       enet_te_params,
                       enet_pe_params,
                       
                       # Time
                       elapsed_time_mins(tic,time.time())])
    
    
resultados = pd.DataFrame(resultados,columns=columns)
resultados.to_csv('./results_regression/resultados.csv',index=False)



+--------+---------+---------+--------+
| Data   | Model   | Train   | Test   |
|--------+---------+---------+--------|
+--------+---------+---------+--------+
(1460, 69)
+-------+---------+---------+---------+
| house | enet_te | 18726.2 | 22824.3 |
+-------+---------+---------+---------+
+-------+---------+---------+-------+
| house | enet_pe | 18538.1 | 23154 |
+-------+---------+---------+-------+
+-------+---------+---------+---------+
| house | xgbs_te | 5417.03 | 19809.7 |
+-------+---------+---------+---------+
+-------+---------+---------+---------+
| house | xgbs_pe | 5264.24 | 19525.6 |
+-------+---------+---------+---------+
(47702, 127)
+-------+---------+---------+---------+
| stack | enet_te | 55486.6 | 79632.3 |
+-------+---------+---------+---------+
+-------+---------+---------+---------+
| stack | enet_pe | 49367.7 | 71283.5 |
+-------+---------+---------+---------+
+-------+---------+---------+---------+
| stack | xgbs_te | 35147.1 | 70684.5 |
+-------+---------+---

stack simplified